In [1]:
import numpy as np
import cupy as cu
import cupyx.scipy.ndimage
import json
import matplotlib.pyplot as plt
import zarr

In [2]:
import czifile
_testFile = czifile.CziFile('sample_data/RBC_full_time_series.czi')


In [3]:
# Extract raw metadata
raw_metadata = _testFile.metadata()

# If you want the metadata in dictionary form (easier to navigate), you can use:
metadata_dict = _testFile.metadata(raw=False)
channels_info = metadata_dict['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Channels']


In [4]:
#metadata_dict

In [5]:
image  = _testFile.asarray()[0, 0, :,:,:,0]

In [6]:
#import tifffile
#image = tifffile.imread('sample_data/RBC_lattice.tif')
print(image.shape)

(834, 300, 2048)


In [7]:
depth, height, original_width = image.shape


In [10]:
%%time
import cupy as cp
import cupyx.scipy.ndimage
import numpy as np

# Constants
voxel_size_x_in_microns = 0.1449922  # Microns
voxel_size_y_in_microns = 0.1449922  # Microns
voxel_size_z_in_microns = 0.3        # Microns
deskewing_angle_in_degrees = 30       # Angle in degrees
angle_in_radians = np.radians(deskewing_angle_in_degrees)

# Image dimensions (replace with actual dimensions of your image stack)
depth, original_height, original_width = image.shape

# Function to calculate translation in X based on Z-index
def calculate_translation_in_X(z, angle_in_radians, voxel_size_z_in_microns, voxel_size_x_in_microns):
    # Calculate the physical offset in the X direction for the current Z index
    physical_offset_x = z * voxel_size_z_in_microns * np.tan(angle_in_radians)
    # Convert the physical offset to pixels in the X dimension
    translation_in_X = physical_offset_x / voxel_size_x_in_microns
    return translation_in_X

# Calculate the maximum translation in X for the last slice to determine the new width
max_translation_in_X = calculate_translation_in_X(depth - 1, angle_in_radians, voxel_size_z_in_microns, voxel_size_x_in_microns)
new_width = int(original_height + np.ceil(max_translation_in_X))

# Initialize the de-skewed volume with the new width
#deskewed_volume = cp.zeros((depth, new_width, original_width), dtype=cp.float32)  # Ensure dtype matches your image :: dtype=deskewed_volume.dtype
zarrOutput = zarr.open('deskewed_volume.zarr', mode='w', shape=(depth, new_width, original_width), chunks=(1, new_width, original_width))



# Deskewing process

for z in range(depth):
    # Calculate translation in X for the current slice based on the Z-index
    translation_in_X = calculate_translation_in_X(z, angle_in_radians, voxel_size_z_in_microns, voxel_size_x_in_microns)
    
    # Define the transformation matrix for this slice with the translation factor in the X direction
    transformation_matrix = cp.array([
        [1, 0, 0],
        [0, 1, 0],
        [translation_in_X, 0, 1]  # Translation in the X direction
    ])
    
       # Calculate the new X index range after the shift
    start_x_index = int(np.ceil(translation_in_X))
    end_x_index = start_x_index + original_height
    
    # Place the shifted image into the de-skewed volume
    #deskewed_volume[z, start_x_index:end_x_index, :] = cupyx.scipy.ndimage.shift(cp.asarray(image[z, :, :]), [translation_in_X, 0]) 
    cupySlice = cp.zeros((new_width, original_width), dtype=cp.float32)
    cupySlice[start_x_index:end_x_index, :] = cupyx.scipy.ndimage.shift(cp.asarray(image[z, :, :]),
                                                                        [translation_in_X, 0]) 

    #numpy_slice = cp.asnumpy(deskewed_volume[z])
    zarrOutput[z] = cp.asnumpy(cupySlice.get())


CPU times: user 17.4 s, sys: 1.85 s, total: 19.2 s
Wall time: 10.4 s


In [ ]:
import tifffile
tifffile.imwrite('Output.tif', zarrOutput)

In [ ]:
import napari
views = napari.Viewer()

views.add_image(deskewed_volume.get(), scale=(.3, .15,.15), name='deskewed')
views.add_image(image, scale=(.3, .15,.15))

In [ ]:
views.add_image(rotated_volume.get(), scale=(.3, .15,.15))

In [ ]:
image2= tifffile.imread('sample_data/RBC_lattice.tif')
views.add_image(image2, scale=(.3, .15,.15))

In [ ]:
image_volume = image
angle_in_degrees = 30
y_pixel_size = 0.15
z_pixel_size = 0.3

depth, height, original_width = image_volume.shape

# Calculate the shear factor based on the excitation angle and pixel sizes
angle_in_radians = cu.radians(angle_in_degrees)
shear_factor = cu.tan(angle_in_radians) * (y_pixel_size / z_pixel_size)

# Calculate the new width to accommodate the shearing
max_skew_in_pixels = (depth - 1) * cu.tan(angle_in_radians) * (z_pixel_size / y_pixel_size)
new_width = int(original_width + cu.abs(max_skew_in_pixels))

# Create a new volume to hold the de-skewed images
deskeWed_volume = cu.zeros((depth, height, new_width), dtype=image_volume.dtype)

# Define the 4x4 affine transformation matrix for 3D shearing in y over z
transformation_matrix = cu.array([
    [1, 0, 0, 0],
    [shear_factor.get(), 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Perform the affine transformation on the entire volume
deskeWed_volume = cupyx.scipy.ndimage.affine_transform(
    image_volume,
    transformation_matrix,
    offset=0,
    output_shape=(depth, height, new_width),
    order=1,
    mode='constant',
    cval=0.0
)



In [ ]:
import napari
views = napari.Viewer()
views.add_image(image_volume.get(), scale=(.3, .15,.15))
views.add_image(deskeWed_volume.get(), scale=(.3, .15,.15))